In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten,\
BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input,decode_predictions

In [ ]:
train_path = '../week9/train/'
validation_path = '../week9/validation/'
test_path = '../week9/test/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,validation_split=0.2) # rescale scales pixel values from 0-255 to 0-1 range

train_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'pants', 'shirt', 'shoes',
                                'shorts', 'skirt', 't-shirt'], batch_size=64,subset='training')

validation_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'pants', 'shirt', 'shoes',
                                'shorts', 'skirt', 't-shirt'], batch_size=64,subset='validation')

test_batches = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(\
directory=test_path, target_size=(224,224), classes=['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'pants', 'shirt', 'shoes',
                                'shorts', 'skirt', 't-shirt'], batch_size=64, shuffle=False)

In [ ]:
X = []
y = []
for features, label in train_batches:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)
#normalize the data
X = X/255.0
y = np.array(y)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    
    Flatten(),
    Dense(units=2, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',\
metrics=['accuracy'])

In [ ]:
#model.fit_generator(x= train_batches, validation_data = validation_batches, epochs=5, verbose=1)
hist = model.fit(
     x = train_batches,
     validation_data = validation_batches, 
     epochs = 50)

In [ ]:
# # Display models statistics
# from matplotlib import pyplot as plt
# loss = hist.history['loss']
# val_loss = hist.history['val_loss']
# acc = hist.history['accuracy']
# val_acc = hist.history['val_accuracy']
# epochsr = range(10)
# plt.figure()
# plt.plot(epochsr, loss, 'g', label='Training loss')
# plt.plot(epochsr, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()
# plt.figure()
# plt.plot(epochsr, acc, 'g', label='Training acc')
# plt.plot(epochsr, val_acc, 'b', label='Validation acc')
# plt.title('Accuracy')
# plt.legend()
# plt.show()
# mymodel = hist.model

In [ ]:
# # serialize model
# model.save("model.h5")
# print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import load_model

# load, create and compile model

model = load_model("model.h5")
model.summary()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# you could either use this custom function or the built in function from https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html

In [ ]:
# eg of itertools.product (gives you a Cartesian product of lists in Python)
l1 = ['a', 'b', 'c']
l2 = ['X', 'Y', 'Z']

for comb in itertools.product(l1, l2):
    print(comb)

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
cm = confusion_matrix(y_true=test_batches.classes, \
                      y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
model.evaluate(train_batches),model.evaluate(validation_batches),model.evaluate(test_batches)

In [ ]:
def TestImgPreprocessing(path):
    '''
    Resizes all images in a given file path to (224,224) and MinMax scales 
    Return array of images
    Adapted from https://stackoverflow.com/questions/21517879/
    python-pil-resize-all-images-in-a-folder
    '''
    from PIL import Image
    import os
    dirs = sorted(os.listdir(path))
    images = []
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((224,224), Image.ANTIALIAS)
            imResize = np.asarray(imResize)/255
            images.append(imResize)
    return np.asarray(images)

test_imgs = TestImgPreprocessing('downloaded_imgs/')

In [ ]:
#Custom model test predictions (open downloaded images folder)
predictions = np.round(model.predict(test_imgs))
print(predictions)

In [ ]:
test_imgs.shape